In [2]:
import pandas as pd
import numpy as np 
import json
import csv


import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm_notebook as tqdm

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('averaged_perceptron_tagger')

import spacy
import math

import string
import sys
import random

from collections import Counter
from itertools import chain

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
!pip install spacy-transformers
!python -m spacy download en_core_web_trf
nlp = spacy.load("en_web_core_trf")



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aarunku5/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


     |████████████████████████████████| 460.2 MB 20 kB/s              


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


OSError: [E050] Can't find model 'en_web_core_trf'. It doesn't seem to be a Python package or a valid path to a data directory.

In [5]:
df_wino = pd.read_csv('./Data/TF_Old.csv')
nlp = spacy.load("en_trf_bertbaseuncased_lg")
words=pd.DataFrame()
frames=[df_wino['sentence1'],df_wino['sentence2']]
words['full'] = pd.concat(frames)
words.reset_index(inplace = True) 
words['casewords'] = words['full'].str.lower()
words['wsw'] = words['casewords'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
words['nopunc'] = words['wsw'].str.replace("'", "")
words['nopunc'] = words['nopunc'].str.replace(".", "")
words['nopunc'] = words['nopunc'].str.replace(",", "")
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
words['text_lemmatized'] = words.nopunc.apply(lemmatize_text)
words['final']="NaN"
def final(x):
    makeitastring = ' '.join(map(str, x))
    makeitastring=makeitastring.replace(",","")
    makeitastring=makeitastring.replace("'","")
    makeitastring=makeitastring.replace("[","")
    makeitastring=makeitastring.replace("]","")
    return makeitastring
for i in tqdm(range(len(words))):    
    column=['text_lemmatized']
    row=[i]
    words.loc[i,'final']=final(words.loc[i,'text_lemmatized'])
words['fullnopunc'] = words['full'].str.replace("'", "")
words['fullnopunc'] = words['fullnopunc'].str.replace(".", "")
words['fullnopunc'] = words['fullnopunc'].str.replace(",", "")
words['label']='NaN'
for i in tqdm(range(len(df_wino))):
    words.loc[i,'label']=df_wino.loc[i,'gold_label']
    words.loc[(i+len(df_wino)),'label']=df_wino.loc[i,'gold_label']
words.to_csv('./test_good_101_1_words.csv', index = False)
e=pd.DataFrame(columns=['label','final','fullnopunc'])
n=pd.DataFrame(columns=['label','final','fullnopunc'])
c=pd.DataFrame(columns=['label','final','fullnopunc'])
#split test labels
for i in tqdm(range(len(words))):
    l=words.loc[i,'label']
    new_row={'label':words.loc[i,'label'],'final':words.loc[i,'final'],'fullnopunc':words.loc[i,'fullnopunc']}
    if(l=='entailment'):
        e=e.append(new_row, ignore_index=True)
    elif(l=='neutral'):
        n=n.append(new_row, ignore_index=True)
    else:
        c=c.append(new_row,ignore_index=True)
e.to_csv('./test_good_101_1_wordse.csv', index = False)
n.to_csv('./test_good_101_1_wordsn.csv', index = False)
c.to_csv('./test_good_101_1_wordsc.csv', index = False)











In [6]:
print(len(e))
print(len(n))
print(len(c))
print(len(df_wino))
print(len(words))

48
104
50
101
202


In [7]:
gwords=pd.read_csv("./test_good_101_1_words.csv")
size=len(gwords)
WSIML=0.5
sentence=gwords['fullnopunc']
lists=[]
vals=[]
for x in tqdm(range(len(sentence))):
    arr=[]
    s=sentence[x]
    word = s.split()
    length=len(word)
    sl=0
    for i in (range(length)):
        sm=0
        for j in (range(length)):
            if(i!=j):
                sm=sm+nlp(word[i]).similarity(nlp(word[j]))
        sm=sm/length
        arr.append(sm)
        sm=abs(sm-WSIML)
        sl=sl+sm
    lists.append(arr)
    vals.append(size/sl)

len(gwords)/pd.Series(vals).sum()

KeyboardInterrupt: 

In [4]:
df_testgood=pd.read_csv("./test_good_101_1.csv")

SIML=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
arr=[]
df = pd.DataFrame(columns=['p', 'h'], data=df_testgood[['sentence1','sentence2']].values)
df['p'] = df['p'].str.replace("'", "")
df['p'] = df['p'].str.replace(".", "")
df['p'] = df['p'].str.replace(",", "")
df['h'] = df['h'].str.replace("'", "")
df['h'] = df['h'].str.replace(".", "")
df['h'] = df['h'].str.replace(",", "")

for x in tqdm(range(len(SIML))):
    dr=0
    for i in tqdm(range(len(df_testgood))):        
        dr=dr+abs(((nlp(df.loc[i,'p'])).similarity(nlp(df.loc[i,'h'])))-SIML[x])
    arr.append(len(df_testgood)/dr)
print(arr)
    
#-------------------------------------------------------------------------------------------------------

garr=[]

df = pd.DataFrame(columns=['p', 'h'], data=df_testgood[['sentence1','sentence2']].values)
df['p'] = df['p'].str.replace("'", "")
df['p'] = df['p'].str.replace(".", "")
df['p'] = df['p'].str.replace(",", "")
df['h'] = df['h'].str.replace("'", "")
df['h'] = df['h'].str.replace(".", "")
df['h'] = df['h'].str.replace(",", "")

for i in tqdm(range(len(df_testgood))):         
    garr.append(abs((len((df.iloc[i]['p']).split()))-(len((df.iloc[i]['h']).split()))))
print(garr)
print(pd.Series(garr).std()/len(df_testgood))    
print(len(garr)/(pd.Series(garr).sum()+1))

#-------------------------------------------------------------------------------------------------------

gsarr=[]
df = pd.DataFrame(columns=['p', 'h'], data=df_testgood[['sentence1','sentence2']].values)
df['p'] = df['p'].str.replace("'", "")
df['p'] = df['p'].str.replace(".", "")
df['p'] = df['p'].str.replace(",", "")
df['h'] = df['h'].str.replace("'", "")
df['h'] = df['h'].str.replace(".", "")
df['h'] = df['h'].str.replace(",", "")

for i in tqdm(range(len(df_testgood))):        
    gsarr.append(abs(((nlp(df.loc[i,'p'])).similarity(nlp(df.loc[i,'h'])))))
print(gsarr)
    
print(pd.Series(gsarr).std()/len(df_testgood))



















[1.5078036884658743, 1.7755167605086704, 2.158818625811296, 2.75318032167654, 3.774922918891222, 5.759273105786393, 9.559867544343893, 11.618984979000448, 7.11004329237453]



[1, 9, 4, 6, 2, 6, 1, 8, 15, 4, 1, 7, 11, 1, 5, 4, 12, 3, 8, 11, 11, 11, 2, 3, 3, 4, 2, 19, 1, 7, 18, 6, 2, 8, 2, 0, 6, 5, 4, 6, 31, 38, 5, 16, 4, 2, 5, 33, 12, 16, 16, 4, 5, 10, 1, 2, 9, 8, 3, 5, 6, 1, 4, 0, 2, 1, 18, 9, 5, 2, 9, 1, 4, 6, 5, 0, 17, 6, 19, 16, 2, 2, 9, 7, 13, 7, 1, 4, 18, 3, 1, 2, 10, 6, 7, 1, 7, 8, 10, 7, 3]
0.06756992585061462
0.13950276243093923



[0.731594078373792, 0.7071323328916573, 0.6084232871713664, 0.83258588081206, 0.8659693743389781, 0.8373076248945541, 0.785089759464352, 0.711765019821002, 0.7204732725577601, 0.7578399374810542, 0.8200743273283415, 0.8207343656307879, 0.7751139141883853, 0.8816582624539723, 0.8842817887763237, 0.9313493167365009, 0.7348446484908576, 0.6534945662526068, 0.673514136586978, 0.6552793109798888, 0.8283842707943571, 0.8180516428091832, 0.7768818276487424, 0.6008899534498305, 0.6728583850337511, 0.726095322785707, 0.8581615607466013, 0.7653848492190759, 0.9160845289055501, 0.8116007957688067, 0.684753683096059, 0.7966191692278501, 0.8166835717099845, 0.8084913075183292, 0.7762492988386241, 0.6970762385609213, 0.5681490553821698, 0.8961349735801857, 0.8999271642928709, 0.5096694766383505, 0.7170249399407199, 0.7647675868650998, 0.8735501884536386, 0.7822725363055315, 0.6545521273466435, 0.7270936806490413, 0.8528961190603768, 0.6776162638926507, 0.5409919781502396, 0.6502575025945189, 0.6241

In [8]:
df_testgood=pd.read_csv("/Users/aarunku5/Desktop/Case_3/Sentences.csv")

SIML=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
arr=[]
df = pd.DataFrame(columns=['p', 'h'], data=df_testgood[['sentence1','sentence2']].values)
df['p'] = df['p'].str.replace("'", "")
df['p'] = df['p'].str.replace(".", "")
df['p'] = df['p'].str.replace(",", "")
df['h'] = df['h'].str.replace("'", "")
df['h'] = df['h'].str.replace(".", "")
df['h'] = df['h'].str.replace(",", "")
df


,p,h
0,A woman in a green shirt preparing to run on a...,A woman is preparing to sleep on a treadmill
1,The dog is catching a treat,the cat is not catching a treat
2,Three young men are watching a tennis match on...,Three young men watching a tennis match on a s...
3,A girl dressed in a pink shirt jeans and flip-...,A funny person in a shirt
4,A man in a green apron smiles behind a food stand,A man smiles
5,A little girl with a hat sits between a womans...,The girl is wearing a hat
6,People are throwing tomatoes at each other,The people are having a food fight
7,A man poses for a photo in front of a Chinese ...,The man is prepared for his photo
8,An older gentleman speaking at a podium,A man giving a speech
9,A man poses for a photo in front of a Chinese ...,The man has experience in taking photos


In [9]:
df_wino = pd.read_csv('/Users/aarunku5/Desktop/Case_3/Sentences.csv')
nlp = spacy.load("en_trf_bertbaseuncased_lg")
words=pd.DataFrame()
frames=[df_wino['sentence1'],df_wino['sentence2']]
words['full'] = pd.concat(frames)
words.reset_index(inplace = True) 
words['casewords'] = words['full'].str.lower()
words['wsw'] = words['casewords'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
words['nopunc'] = words['wsw'].str.replace("'", "")
words['nopunc'] = words['nopunc'].str.replace(".", "")
words['nopunc'] = words['nopunc'].str.replace(",", "")
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
words['text_lemmatized'] = words.nopunc.apply(lemmatize_text)
words['final']="NaN"
def final(x):
    makeitastring = ' '.join(map(str, x))
    makeitastring=makeitastring.replace(",","")
    makeitastring=makeitastring.replace("'","")
    makeitastring=makeitastring.replace("[","")
    makeitastring=makeitastring.replace("]","")
    return makeitastring
for i in tqdm(range(len(words))):    
    column=['text_lemmatized']
    row=[i]
    words.loc[i,'final']=final(words.loc[i,'text_lemmatized'])
words['fullnopunc'] = words['full'].str.replace("'", "")
words['fullnopunc'] = words['fullnopunc'].str.replace(".", "")
words['fullnopunc'] = words['fullnopunc'].str.replace(",", "")

words

,index,full,casewords,wsw,nopunc,text_lemmatized,final,fullnopunc
0,0,"A woman, in a green shirt, preparing to run on...","a woman, in a green shirt, preparing to run on...","woman, green shirt, preparing run treadmill.",woman green shirt preparing run treadmill,"[woman, green, shirt, preparing, run, treadmill]",woman green shirt preparing run treadmill,A woman in a green shirt preparing to run on a...
1,1,The dog is catching a treat.,the dog is catching a treat.,dog catching treat.,dog catching treat,"[dog, catching, treat]",dog catching treat,The dog is catching a treat
2,2,Three young men are watching a tennis match on...,three young men are watching a tennis match on...,three young men watching tennis match large sc...,three young men watching tennis match large sc...,"[three, young, men, watching, tennis, match, l...",three young men watching tennis match large sc...,Three young men are watching a tennis match on...
3,3,"A girl dressed in a pink shirt, jeans, and fli...","a girl dressed in a pink shirt, jeans, and fli...","girl dressed pink shirt, jeans, flip-flops sit...",girl dressed pink shirt jeans flip-flops sitti...,"[girl, dressed, pink, shirt, jean, flip-flop, ...",girl dressed pink shirt jean flip-flop sitting...,A girl dressed in a pink shirt jeans and flip-...
4,4,A man in a green apron smiles behind a food st...,a man in a green apron smiles behind a food st...,man green apron smiles behind food stand.,man green apron smiles behind food stand,"[man, green, apron, smile, behind, food, stand]",man green apron smile behind food stand,A man in a green apron smiles behind a food stand
5,5,A little girl with a hat sits between a woman'...,a little girl with a hat sits between a woman'...,little girl hat sits woman's feet sand front p...,little girl hat sits womans feet sand front pa...,"[little, girl, hat, sits, woman, foot, sand, f...",little girl hat sits woman foot sand front pai...,A little girl with a hat sits between a womans...
6,6,People are throwing tomatoes at each other.,people are throwing tomatoes at each other.,people throwing tomatoes other.,people throwing tomatoes other,"[people, throwing, tomato, other]",people throwing tomato other,People are throwing tomatoes at each other
7,7,A man poses for a photo in front of a Chinese ...,a man poses for a photo in front of a chinese ...,man poses photo front chinese building jumping.,man poses photo front chinese building jumping,"[man, pose, photo, front, chinese, building, j...",man pose photo front chinese building jumping,A man poses for a photo in front of a Chinese ...
8,8,An older gentleman speaking at a podium.,an older gentleman speaking at a podium.,older gentleman speaking podium.,older gentleman speaking podium,"[older, gentleman, speaking, podium]",older gentleman speaking podium,An older gentleman speaking at a podium
9,9,A man poses for a photo in front of a Chinese ...,a man poses for a photo in front of a chinese ...,man poses photo front chinese building jumping.,man poses photo front chinese building jumping,"[man, pose, photo, front, chinese, building, j...",man pose photo front chinese building jumping,A man poses for a photo in front of a Chinese ...


In [10]:
def final(x):
    makeitastring = ' '.join(map(str, x))
    makeitastring=makeitastring.replace(",","")
    makeitastring=makeitastring.replace("'","")
    makeitastring=makeitastring.replace("[","")
    makeitastring=makeitastring.replace("]","")
    return makeitastring

for i in tqdm(range(len(words))):    
    column=['text_lemmatized']
    row=[i]
    words.loc[i,'final']=final(words.loc[i,'text_lemmatized'])


arr=[]
for i in tqdm(range(len(df))):
    s=words.iloc[i]['final']
    s=s.lower()
    slist=s.split()
    slist = list(dict.fromkeys(slist))
    ss=words.iloc[i+12]['final']
    sslist=ss.split()
    sslist = list(dict.fromkeys(sslist))
    s=final(slist)
    ss=final(sslist)
    slist=s.split()
    print(s)
    print(ss)
    c=0
    for j in range(len(slist)):
         if(slist[j] in ss):
                c=c+1
    arr.append(c)

print(arr)


woman green shirt preparing run treadmill
woman preparing sleep treadmill
dog catching treat
cat catching treat
three young men watching tennis match large screen outdoors
three young men watching tennis match screen outdoors brother playing
girl dressed pink shirt jean flip-flop sitting playing lollipop machine
funny person shirt
man green apron smile behind food stand
man smile
little girl hat sits woman foot sand front pair colorful tent
girl wearing hat
people throwing tomato other
people food fight
man pose photo front chinese building jumping
man prepared photo
older gentleman speaking podium
man giving speech
man pose photo front chinese building jumping
man experience taking photo
people waiting line food vendor
people sit wait order nice restaurant
number 13 kick soccer ball towards goal childrens game
player passing ball soccer game

[3, 2, 8, 1, 2, 2, 1, 2, 0, 2, 1, 3]


In [11]:
def final(x):
    makeitastring = ' '.join(map(str, x))
    makeitastring=makeitastring.replace(",","")
    makeitastring=makeitastring.replace("'","")
    makeitastring=makeitastring.replace("[","")
    makeitastring=makeitastring.replace("]","")
    return makeitastring



arr=[]
arr1=[]
arr2=[]
arr3=[]
for i in tqdm(range(len(df))):
    s=words.iloc[i]['final']
    s=s.lower()
    slist=s.split()
    sxlist = list(dict.fromkeys(slist))
    ss=words.iloc[i+12]['final']
    sslist=ss.split()
    ssxlist = list(dict.fromkeys(sslist))
    sx=final(sxlist)
    ssx=final(ssxlist)
    sxlist=sx.split()
    hl=len(sxlist)
    pl=len(ssxlist)
    c=0
    for j in range(len(sxlist)):
         if(sxlist[j] in ssx):
                c=c+1
    if(c==0):
        c=c+0.1
    arr.append(c)
    arr1.append(hl/c)
    arr2.append(pl/c)
    arr3.append((hl+pl)/c)

print(arr)
print(arr1)
print(arr2)
print(arr3)







[3, 2, 8, 1, 2, 2, 1, 2, 0.1, 2, 1, 3]
[2.0, 1.5, 1.125, 10.0, 3.5, 5.5, 4.0, 3.5, 40.0, 3.5, 5.0, 3.0]
[1.3333333333333333, 1.5, 1.25, 3.0, 1.0, 1.5, 3.0, 1.5, 30.0, 2.0, 6.0, 1.6666666666666667]
[3.3333333333333335, 3.0, 2.375, 13.0, 4.5, 7.0, 7.0, 5.0, 70.0, 5.5, 11.0, 4.666666666666667]


In [12]:
fcount = arr[:6]
lcount= arr[6:]
print(fcount)
print(pd.Series(fcount).mean())
print(lcount)
print(pd.Series(lcount).mean())

[3, 2, 8, 1, 2, 2]
3.0
[1, 2, 0.1, 2, 1, 3]
1.5166666666666666


In [13]:
fcount = arr1[:6]
lcount= arr1[6:]
print(fcount)
print(pd.Series(fcount).mean())
print(lcount)
print(pd.Series(lcount).mean())

[2.0, 1.5, 1.125, 10.0, 3.5, 5.5]
3.9375
[4.0, 3.5, 40.0, 3.5, 5.0, 3.0]
9.833333333333334


In [14]:
fcount = arr2[:6]
lcount= arr2[6:]
print(fcount)
print(pd.Series(fcount).mean())
print(lcount)
print(pd.Series(lcount).mean())

[1.3333333333333333, 1.5, 1.25, 3.0, 1.0, 1.5]
1.597222222222222
[3.0, 1.5, 30.0, 2.0, 6.0, 1.6666666666666667]
7.361111111111111


In [15]:
fcount = arr3[:6]
lcount= arr3[6:]
print(fcount)
print(pd.Series(fcount).mean())
print(lcount)
print(pd.Series(lcount).mean())

[3.3333333333333335, 3.0, 2.375, 13.0, 4.5, 7.0]
5.534722222222222
[7.0, 5.0, 70.0, 5.5, 11.0, 4.666666666666667]
17.194444444444446


In [16]:
def final(x):
    makeitastring = ' '.join(map(str, x))
    makeitastring=makeitastring.replace(",","")
    makeitastring=makeitastring.replace("'","")
    makeitastring=makeitastring.replace("[","")
    makeitastring=makeitastring.replace("]","")
    return makeitastring



arr1=[]
for i in tqdm(range(6)):
    s=words.iloc[i]['fullnopunc']
    s=s.lower()
    slist=s.split()
    sxlist = list(dict.fromkeys(slist))
    ss=words.iloc[i+12]['fullnopunc']
    sslist=ss.split()
    ssxlist = list(dict.fromkeys(sslist))
    sx=final(sxlist)
    ssx=final(ssxlist)
    sxlist=sx.split()
    hl=len(sxlist)
    pl=len(ssxlist)
    print(ssx)
    print(sx)
    c=0
    for j in tqdm(range(len(sxlist))):
        arr=[]
        for k in range(len(ssxlist)):
            arr.append(nlp(sxlist[j]).similarity(nlp(ssxlist[k])))
        arr1.append(pd.Series(arr).max())







A woman is preparing to sleep on a treadmill
a woman in green shirt preparing to run on treadmill



the cat is not catching a treat
the dog is catching a treat



Three young men watching a tennis match on screen outdoors because their brother is playing
three young men are watching a tennis match on large screen outdoors



A funny person in a shirt
a girl dressed in pink shirt jeans and flip-flops sitting down playing with lollipop machine



A man smiles
a man in green apron smiles behind food stand



The girl is wearing a hat
a little girl with hat sits between womans feet in the sand front of pair colorful tents


In [17]:
len(arr1)

69

In [18]:
s1=arr1[:6]
s2=arr1[6:9]
s3=arr1[9:18]
s4=arr1[18:28]
s5=arr1[28:35]
s6=arr1[35:]
print(len(s1))
print(len(s2))
print(len(s3))
print(len(s4))
print(len(s5))
print(len(s6))



6
3
9
10
7
34


In [19]:
print(pd.Series(s1).sum())
print(pd.Series(s2).sum())
print(pd.Series(s3).sum())
print(pd.Series(s4).sum())
print(pd.Series(s5).sum())
print(pd.Series(s6).sum())

pd.Series(arr1).sum()/6

5.475317951563026
2.7865678392603814
8.900851224481983
9.871537398759761
6.520272632857348
29.03587710788107


10.431737359133928

In [241]:
def final(x):
    makeitastring = ' '.join(map(str, x))
    makeitastring=makeitastring.replace(",","")
    makeitastring=makeitastring.replace("'","")
    makeitastring=makeitastring.replace("[","")
    makeitastring=makeitastring.replace("]","")
    return makeitastring



arr1=[]
for i in tqdm(range(6,12)):
    s=words.iloc[i]['fullnopunc']
    s=s.lower()
    slist=s.split()
    sxlist = list(dict.fromkeys(slist))
    ss=words.iloc[i+12]['fullnopunc']
    sslist=ss.split()
    ssxlist = list(dict.fromkeys(sslist))
    sx=final(sxlist)
    ssx=final(ssxlist)
    sxlist=sx.split()
    hl=len(sxlist)
    pl=len(ssxlist)
    print(sx)
    print(ssx)
    c=0
    for j in tqdm(range(len(sxlist))):
        arr=[]
        for k in range(len(ssxlist)):
            arr.append(nlp(sxlist[j]).similarity(nlp(ssxlist[k])))
        arr1.append(pd.Series(arr).max())








people are throwing tomatoes at each other
The people are having a food fight



a man poses for photo in front of chinese building by jumping
The man is prepared for his photo



an older gentleman speaking at a podium
A man giving a speech



a man poses for photo in front of chinese building by jumping
The man has experience in taking photos



people are waiting in line by a food vendor
People sit and wait for their orders at a nice down restaurant



number 13 kicks a soccer ball towards the goal during childrens game
A player passing the ball in a soccer game


In [242]:
len(arr1)

59

In [243]:
s1=arr1[:4]
s2=arr1[4:11]
s3=arr1[11:15]
s4=arr1[15:22]
s5=arr1[22:27]
s6=arr1[27:]
print(len(s1))
print(len(s2))
print(len(s3))
print(len(s4))
print(len(s5))
print(len(s6))




4
7
4
7
5
32


In [244]:
print(pd.Series(s1).sum())
print(pd.Series(s2).sum())
print(pd.Series(s3).sum())
print(pd.Series(s4).sum())
print(pd.Series(s5).sum())
print(pd.Series(s6).sum())

pd.Series(arr1).sum()/6

3.6143740579938752
6.533509352985964
3.6679523270315877
6.058359412793398
4.355809793918845
28.580663467190597


8.801778068652379

0             woman green shirt preparing run treadmill
1                                    dog catching treat
2     three young men watching tennis match large sc...
3     girl dressed pink shirt jean flip-flop sitting...
4               man green apron smile behind food stand
5     little girl hat sits woman foot sand front pai...
6                          people throwing tomato other
7         man pose photo front chinese building jumping
8                       older gentleman speaking podium
9         man pose photo front chinese building jumping
10                      people waiting line food vendor
11    number 13 kick soccer ball towards goal childr...
12                      woman preparing sleep treadmill
13                                   cat catching treat
14    three young men watching tennis match screen o...
15                                   funny person shirt
16                                            man smile
17                                     girl wear